# مثال على وكيل حجز الفنادق والرحلات الجوية

هذا الحل سيساعدك في حجز تذاكر الطيران والفنادق. السيناريو هو رحلة من مطار لندن هيثرو LHR في 20 فبراير 2024 إلى مطار نيويورك JFK مع العودة في 27 فبراير 2025 على الدرجة الاقتصادية مع الخطوط الجوية البريطانية فقط. أرغب في الإقامة في فندق هيلتون في نيويورك، يرجى توفير تكاليف الرحلة والفندق.


# تهيئة خدمة Azure AI Agent والحصول على معلومات التكوين من **.env**

### **.env**

قم بإنشاء ملف .env

يحتوي ملف **.env** على سلسلة الاتصال الخاصة بخدمة Azure AI Agent، النموذج المستخدم من قبل AOAI، وخدمة Google API Search المقابلة، وENDPOINT، وغيرها.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "اسم نشر نموذج خدمة Azure AI Agent الخاص بك"

[**NOTE**] ستحتاج إلى نموذج بمعدل حد 100,000 (رموز في الدقيقة) وحد معدل 600 (طلبات في الدقيقة).

  يمكنك الحصول على النموذج من Azure AI Foundry - النموذج وENDPOINT.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "سلسلة الاتصال الخاصة بمشروع خدمة Azure AI Agent الخاص بك"

  يمكنك الحصول على سلسلة الاتصال الخاصة بالمشروع من شاشة نظرة عامة على المشروع في بوابة AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "مفتاح API الخاص بخدمة SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "نقطة النهاية الخاصة بخدمة SERPAPI Search"

للحصول على اسم نشر النموذج وسلسلة الاتصال الخاصة بمشروع خدمة Azure AI Agent، تحتاج إلى إنشاء خدمة Azure AI Agent. يُوصى باستخدام [هذا القالب](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) لإنشائها مباشرة. (***ملاحظة:*** خدمة Azure AI Agent متوفرة حاليًا في مناطق محدودة. يُوصى بالرجوع إلى [هذا الرابط](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) لتحديد المنطقة).

يحتاج الوكيل إلى الوصول إلى SERPAPI. يُوصى بالتسجيل باستخدام [هذا الرابط](https://serpapi.com/searches). بعد التسجيل، يمكنك الحصول على مفتاح API فريد ونقطة نهاية.


# الإعداد

لتشغيل هذا الدفتر، ستحتاج إلى التأكد من تثبيت المكتبات المطلوبة عن طريق تشغيل الأمر `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

يجب أن تكون نسخة النواة الدلالية لديك على الأقل 1.27.2.


يرجى تحميل إعدادات وموارد ملف .env الخاص بك والتأكد من أنك قد أضفت المفاتيح والإعدادات وأنشأت ملف .env محلي.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# تسجيل الدخول إلى Azure

تحتاج الآن إلى تسجيل الدخول إلى Azure. افتح نافذة طرفية وقم بتشغيل الأمر التالي:

```bash
az login
```

سيطلب منك هذا الأمر إدخال بيانات اعتماد Azure الخاصة بك، مما يتيح لخدمة Azure AI Agent العمل بشكل صحيح.


# شرح:
هذه متغير يخزن مفتاح API للوصول إلى خدمة API الخاصة بصفحة نتائج محرك البحث (SERP). مفتاح API هو معرف فريد يُستخدم لمصادقة الطلبات المرتبطة بحسابك.

الغرض: الغرض من هذا السطر هو تخزين مفتاح API في متغير بحيث يمكن استخدامه لمصادقة الطلبات إلى خدمة SERP API. مفتاح API مطلوب للوصول إلى الخدمة وإجراء عمليات البحث.

كيفية الحصول على مفتاح SERP API: للحصول على مفتاح SERP API، اتبع هذه الخطوات العامة على https://serpapi.com (قد تختلف الخطوات الدقيقة حسب خدمة SERP API المحددة التي تستخدمها):

اختر خدمة SERP API: هناك العديد من خدمات SERP API المتاحة، مثل SerpAPI، وGoogle Custom Search JSON API، وغيرها. اختر الخدمة التي تناسب احتياجاتك.

سجل للحصول على حساب: انتقل إلى موقع الخدمة المختارة وقم بالتسجيل للحصول على حساب. قد تحتاج إلى تقديم بعض المعلومات الأساسية والتحقق من عنوان بريدك الإلكتروني.

أنشئ مفتاح API: بعد التسجيل، قم بتسجيل الدخول إلى حسابك وانتقل إلى قسم API أو لوحة التحكم. ابحث عن خيار لإنشاء أو توليد مفتاح API جديد.
انسخ مفتاح API إلى ملف .env الخاص بك.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# الشرح:
BASE_URL: هذا متغير يخزن عنوان URL الأساسي لنقطة النهاية الخاصة بـ SERP API. اسم المتغير BASE_URL هو اصطلاح يُستخدم للإشارة إلى أن هذا العنوان هو نقطة البداية لإجراء طلبات API.

'https://serpapi.com/search':

هذا هو النص الفعلي لعنوان URL الذي يتم تعيينه لمتغير BASE_URL. يمثل نقطة النهاية لإجراء استعلامات البحث باستخدام SERP API.

# الهدف:
الهدف من هذا السطر هو تعريف ثابت يحتوي على عنوان URL الأساسي لـ SERP API. سيتم استخدام هذا العنوان كنقطة انطلاق لبناء طلبات API لإجراء عمليات البحث.

# الاستخدام:
من خلال تعريف عنوان URL الأساسي في متغير، يمكنك إعادة استخدامه بسهولة في جميع أنحاء الكود الخاص بك كلما احتجت إلى إجراء طلبات إلى SERP API. هذا يجعل الكود الخاص بك أكثر قابلية للصيانة ويقلل من خطر الأخطاء الناتجة عن كتابة العنوان بشكل صريح في أماكن متعددة. المثال الحالي هو https://serpapi.com/search?engine=bing الذي يستخدم Bing API للبحث. يمكن اختيار API مختلف من خلال https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# شرح:

هذا هو المكان الذي يوجد فيه كود الإضافة الخاص بك.

تعريف الفئة: `class BookingPlugin`: تعريف فئة باسم BookingPlugin تحتوي على طرق لحجز الفنادق والرحلات الجوية.

طريقة حجز الفنادق:

- `@kernel_function(description="booking hotel")`: ديكور يصف الوظيفة كوظيفة أساسية لحجز الفنادق.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: تعريف طريقة لحجز الفنادق مع معلمات مشروحة ونوع الإرجاع.

تقوم الطريقة بإنشاء قاموس للمعلمات الخاصة بطلب حجز الفندق وترسل طلب GET إلى واجهة برمجة التطبيقات SERP. تتحقق من حالة الاستجابة وتعيد خصائص الفندق إذا نجح الطلب، أو تعيد None إذا فشل الطلب.

طريقة حجز الرحلات الجوية:

- `@kernel_function(description="booking flight")`: ديكور يصف الوظيفة كوظيفة أساسية لحجز الرحلات الجوية.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: تعريف طريقة لحجز الرحلات الجوية مع معلمات مشروحة ونوع الإرجاع.

تقوم الطريقة بإنشاء قواميس للمعلمات الخاصة بطلبات الرحلات المغادرة والعودة وترسل طلبات GET إلى واجهة برمجة التطبيقات SERP. تتحقق من حالة الاستجابة وتضيف معلومات الرحلة إلى سلسلة النتائج إذا نجح الطلب، أو تطبع رسالة خطأ إذا فشل الطلب. تعيد الطريقة سلسلة النتائج التي تحتوي على معلومات الرحلة.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# الشرح:
استيراد المكتبات: يتم استيراد الوحدات اللازمة لاعتماديات Azure، ووكيل الذكاء الاصطناعي، ومحتوى رسائل الدردشة، ودور المؤلف، ومُزخرف وظائف النواة.

مدير السياق غير المتزامن: 
`async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: 
يُعد هذا الإعداد مدير سياق غير متزامن للتعامل مع اعتماديات Azure وإنشاء عميل لوكيل الذكاء الاصطناعي.

اسم الوكيل والتعليمات:
- `AGENT_NAME = "BookingAgent"`: يحدد اسم الوكيل.
- `AGENT_INSTRUCTIONS = """..."""`: يوفر تعليمات مفصلة للوكيل حول كيفية التعامل مع طلبات الحجز.

إنشاء تعريف الوكيل: 
`agent_definition = await client.agents.create_agent(...)`: 
ينشئ تعريفًا للوكيل باستخدام النموذج المحدد، الاسم، والتعليمات.

إنشاء وكيل AzureAI: 
`agent = AzureAIAgent(...)`: 
ينشئ وكيل AzureAI باستخدام العميل، تعريف الوكيل، والمكون الإضافي المحدد.

إنشاء سلسلة: 
`thread: AzureAIAgentThread | None = None`: 
ينشئ سلسلة للوكيل. ليس من الضروري إنشاء سلسلة أولاً - إذا تم توفير القيمة `None`، سيتم إنشاء سلسلة جديدة أثناء أول استدعاء وإرجاعها كجزء من الاستجابة.

مدخلات المستخدم: 
`user_inputs = ["..."]`: 
يحدد قائمة بمدخلات المستخدم التي سيقوم الوكيل بمعالجتها.

في كتلة `finally`، يتم حذف السلسلة والوكيل لتنظيف الموارد.


# المصادقة

تُعد `DefaultAzureCredential` جزءًا من Azure SDK لـ Python. توفر طريقة افتراضية للمصادقة مع خدمات Azure. تحاول المصادقة باستخدام عدة طرق بترتيب محدد، مثل متغيرات البيئة، الهوية المُدارة، وبيانات اعتماد Azure CLI.

العمليات غير المتزامنة: يشير الوحدة aio إلى أن فئة DefaultAzureCredential تدعم العمليات غير المتزامنة. هذا يعني أنه يمكنك استخدامها مع asyncio لتنفيذ طلبات مصادقة غير متزامنة.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
